In [ ]:
######################################
############### CONFIG ###############
######################################

# Enter start and end row here. Note that the first row is row 0 which is the header #
start_row = 1
end_row   = 1500

# Enter file names here. Outputs will appended at the end of this file #
input_file  = "all_visa_2016.csv"
output_file = "output_file_name.csv"

# Enter your API key from Google APIs #
api_key = "your_key_here"

In [ ]:
import re
import json
import pyprind
from urllib.request import urlopen

In [ ]:
n = end_row - start_row + 1
bar = pyprind.ProgBar(n, width=60)

file = open(output_file,'a')
line_cnt = -1

with open(input_file) as f:
    for line in f:
        line_cnt += 1
        if line_cnt < start_row or line_cnt > end_row:
            continue
        line = re.sub('[\"\n]', '', line)
        tmp = line[0:25] + ' ' + line[25:40]
        tmp = tmp.split()
        tmp = "%20".join(tmp)
        url = "https://maps.googleapis.com/maps/api/geocode/json?address=" + tmp + "&key=" + api_key
        ## print(url)
        response = urlopen(url).read().decode('utf8')
        obj = json.loads(response)
        if (obj['status'] == 'OK'):
            lat = obj['results'][0]['geometry']['location']['lat']
            lng = obj['results'][0]['geometry']['location']['lng']
            zipcode = obj['results'][0]['address_components'][-1]['long_name']
            address = obj['results'][0]['formatted_address']
            address = re.sub('[^A-Za-z0-9./\- ]', '', address)
        else:
            lat = -1
            lng = -1
            zipcode = -1
            address = "NaN"
        file.write('\"' + line[:40] + '\",' + line[41:45] + ',' + str(lat) + ',' + str(lng) + ',\"' + str(address) + '\",' + str(zipcode) + '\n')
        bar.update()
        
file.close()